In [1]:
import os
import pickle
import collections
import datetime
import copy
import numpy as np
import scipy as sp
import pandas as pd
import tensorflow as tf
import happybase
import warnings
import sklearn.model_selection
import sklearn.preprocessing
import csv
import json
import pickle

In [2]:
data_dir =  r"/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data"

In [3]:
file_list = []
for root,_,path_list in os.walk(data_dir):
    for path in path_list:
        file_list.append(os.path.join(root,path))
print(file_list)

['/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000000_0', '/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000002_0', '/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000001_0', '/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data/000003_0']


In [4]:
%%time
#读取用户数据
all_file_data = []
for file_name in file_list:
    all_file_data.append(pd.read_csv(os.path.join(data_dir,file_name),sep='\001',header=None,error_bad_lines=False))
df = pd.concat(all_file_data,axis=0,ignore_index=True)
del all_file_data
column_list = ['用户id','用户姓名','用户性别','用户出生时间','所在公司','所处职位','行业','微信网名','微信性别','常驻省','常驻市','是否付费用户','付费等级','关注的作者','作者类型','关注的马甲','关注的公司','文章id','是否精选文章','浏览文章时间','浏览时长','文章频道','文章类型','文章标签','文章图片数量','文章字数','新定位省','新定位市','评论内容','打赏金额','设备型号','使用网络']
df.columns = column_list

b'Skipping line 461777: expected 32 fields, saw 33\n'
<string>:2: DtypeWarning: Columns (1,3,4,5,6,7,15,16,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
<string>:2: DtypeWarning: Columns (1,2,3,4,5,6,7,15,16,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 54.5 s, sys: 2.52 s, total: 57 s
Wall time: 57 s


In [2]:
#
with open("multi_hot_column_odict.pkl","rb") as f:
    multi_hot_column_odict = pickle.load(f)
with open("y_onehot.pkl","rb") as f:
    y_onehot = pickle.load(f)
#with tf.device('/device:GPU:1'):
#    model = tf.keras.models.load_model("model_output")

In [11]:
multi_hot_column_odict.keys()

odict_keys(['关注的马甲', '关注的作者'])

In [3]:
y_onehot.classes_

array(['21~25', '26~29', '30~35', '36~45', '46~50', '50~', '~20'],
      dtype='<U5')

In [13]:
predict_df = df[list(multi_hot_column_odict.keys()) + ["用户id"]]
del df

NameError: name 'df' is not defined

In [14]:
predict_df

,关注的马甲,关注的作者,用户id
0,-1,-1,00001D26FE991A4BC4A01325D5E222DE
1,-1,-1,00006C3EF7F79B0F37EEFB8381505530
2,-1,-1,0000D0E1960239E8362F6B4100D9790A
3,-1,-1,0000f55e60d5ba7b36004429b235c95096f7153d
4,-1,-1,00014E9D5CF948B3FE6376564B23F64F
...,...,...,...
15703260,-1,-1,qlu2xsthqk7tp0mncpgr
15703261,-1,-1,qlu30bysbp1mj150uc1m
15703262,-1,-1,qlu30t7n6f2b1jgzexv
15703263,-1,-1,qlu315b37v5rc273amb0


In [20]:
%%time
#测试数据
warnings.filterwarnings('ignore')
output_csv = r"/mnt1/zhaodachuan/data/predict_user_attribute/data/age.csv"
with open(output_csv, 'w', newline='') as csvfile:
    fieldnames = ['age', 'udid' , "probility"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i,chunk in enumerate(np.array_split(predict_df, 30000)):
        udid_list = chunk["用户id"].values.tolist()
        predict_x = []
        for index in chunk.index:
            temp_x_list = []
            for j,column in enumerate(multi_hot_column_odict):
                split_column_list = chunk.loc[index,column].split(",") if isinstance(chunk.loc[index,column],str) else [str(predict_df.loc[index,column])]
                temp_x_list.extend(multi_hot_column_odict[column].transform([split_column_list]).tolist()[0])
            predict_x.append(temp_x_list)
        y_predict = model.predict([predict_x])
        #print(y_predict)
        #print(len(y_predict[0]))
        #print(len(udid_list))
        for y_predict_i,udid in zip(y_predict[0],udid_list):
            writer.writerow({'age':y_onehot.inverse_transform(np.array([y_predict_i]))[0], 'udid': udid , 'probility':json.dumps(y_predict_i.tolist())})
        if i % 300 == 0:
            print(i)
        #break

0
300
600
900
1200
1500
1800
2100
2400
2700
3000
3300
3600
3900
4200
4500
4800
5100
5400
5700
6000
6300
6600
6900
7200
7500
7800
8100
8400
8700
9000
9300
9600
9900
10200
10500
10800
11100
11400
11700
12000
12300
12600
12900
13200
13500
13800
14100
14400
14700
15000
15300
15600
15900
16200
16500
16800
17100
17400
17700
18000
18300
18600
18900
19200
19500
19800
20100
20400
20700
21000
21300
21600
21900
22200
22500
22800
23100
23400
23700
24000
24300
24600
24900
25200
25500
25800
26100
26400
26700
27000
27300
27600
27900
28200
28500
28800
29100
29400
29700
CPU times: user 12h 52s, sys: 1min 27s, total: 12h 2min 19s
Wall time: 12h 1min 7s


In [21]:
import happybase
output_df = pd.read_csv(output_csv)

In [22]:
output_df

,age,udid,probility
0,30~35,00001D26FE991A4BC4A01325D5E222DE,"[0.09202418476343155, 0.15535157918930054, 0.2..."
1,30~35,00006C3EF7F79B0F37EEFB8381505530,"[0.09202418476343155, 0.15535157918930054, 0.2..."
2,30~35,0000D0E1960239E8362F6B4100D9790A,"[0.09202418476343155, 0.15535157918930054, 0.2..."
3,30~35,0000f55e60d5ba7b36004429b235c95096f7153d,"[0.09202418476343155, 0.15535157918930054, 0.2..."
4,30~35,00014E9D5CF948B3FE6376564B23F64F,"[0.09202418476343155, 0.15535157918930054, 0.2..."
...,...,...,...
15703260,30~35,qlu2xsthqk7tp0mncpgr,"[0.09202418476343155, 0.15535157918930054, 0.2..."
15703261,30~35,qlu30bysbp1mj150uc1m,"[0.09202418476343155, 0.15535157918930054, 0.2..."
15703262,30~35,qlu30t7n6f2b1jgzexv,"[0.09202418476343155, 0.15535157918930054, 0.2..."
15703263,30~35,qlu315b37v5rc273amb0,"[0.09202418476343155, 0.15535157918930054, 0.2..."


In [23]:
output_df["age"].unique()

array(['30~35', '26~29', '36~45', '~20', '21~25', '46~50', '50~'],
      dtype=object)

In [24]:
len(output_df["udid"].unique())

15102652

In [25]:
pool = happybase.ConnectionPool(size=100, host="47.110.125.69", port=9090, timeout=None,
                                protocol='binary'
                                )
with pool.connection() as conn:
    table = conn.table("feature_center:user_profile")
    with table.batch(batch_size=400) as batch:
        for i,index in enumerate(output_df.index):
            batch.put(output_df.loc[index,"udid"].encode(), {"attribute:predict_age".encode(): output_df.loc[index,"age"].encode()})
            #break
            if i % 100000 == 0:
                print(i)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000